### MLP를 이해하기 위해 최대한 풀어서 구현한 코드입니다.

MNIST 파일 다운은 download_image.py 를 먼저 실행 해주세요.

layer는 4개로 구성했으며 직관적으로 볼 수 있게, 절차적으로 하나씩 선언 구현했습니다.

In [11]:
import numpy as np
import os, gzip

### 데이터 로딩 함수:   
MNIST 데이터셋을 불러오는 함수.  
MNIST는 손글씨 숫자(0-9) 이미지 데이터셋으로, 학습을 위해 필요한 데이터. 

In [12]:
def _images(path):
    with gzip.open(path) as f:
        pixels = np.frombuffer(f.read(), 'B', offset=16)
    return pixels.reshape(-1, 28 * 28).astype('float32') / 255  # 28x28 픽셀 이미지를 벡터로 변환 후 정규화


def _labels(path):
    with gzip.open(path) as f:
        integer_labels = np.frombuffer(f.read(), 'B', offset=8)
    return np.eye(10)[integer_labels]  # one-hot 인코딩 적용 (0-9 숫자에 대해 10차원 벡터로 변환)
# 데이터셋 불러오기
dataset_dir = os.path.join(os.getcwd(), 'data')
key_file = {'train_img': 'train-images-idx3-ubyte.gz', 'train_label': 'train-labels-idx1-ubyte.gz',
            'test_img': 't10k-images-idx3-ubyte.gz', 'test_label': 't10k-labels-idx1-ubyte.gz'}
X_train, y_train = _images(os.path.join(dataset_dir, key_file['train_img'])), _labels(
    os.path.join(dataset_dir, key_file['train_label']))
X_test, y_test = _images(os.path.join(dataset_dir, key_file['test_img'])), _labels(
    os.path.join(dataset_dir, key_file['test_label']))


## 활성화 함수 (시그모이드)
### 출력되는 그래프
x = 0일 때 σ(0) = 0.5
x가 작을수록 값이 0에 가까워짐
x가 클수록 값이 1에 가까워짐

### 시그모이드 함수의 특징
✅ 장점
- 출력값이 (0,1) 범위 → 확률 해석 가능
- 비선형 함수 → 딥러닝에서 비선형성을 학습할 수 있음

❌ 단점
기울기 소실(Vanishing Gradient) 문제
- x 값이 너무 크거나 작을 때 미분 값이 0에 가까워져 학습이 어려움
- 이를 해결하기 위해 ReLU 함수 같은 다른 활성화 함수가 주로 사용됨

In [13]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # 비선형성을 부여하여 모델이 복잡한 패턴을 학습할 수 있도록 함

def sigmoid_prime(x):
    return x * (1 - x)  # 시그모이드의 미분값: 역전파 시 그래디언트 계산에 필요

### 완전 연결층(Fully Connected Layer, FC Layer)
➡ 신경망에서 입력 뉴런과 출력 뉴런을 연결하고, 가중치(W)와 편향(b)을 저장합니다.

In [14]:
class FCLayer:
    def __init__(self, input_size, output_size):
        """
        - W는 입력 뉴런 수(input_size) × 출력 뉴런 수(output_size) 크기의 행렬.
        - np.random.randn()을 사용하여 정규분포(평균=0, 표준편차=1)에서 난수를 샘플링.
        - * 0.01을 곱해 가중치를 작은 값으로 초기화하여 학습 안정성 향상.
        example)
        W = np.random.randn(3, 2) * 0.01  # 3×2 크기의 가중치 행렬 생성
        초기화된 W: [[ 0.0072 -0.0023]
                    [-0.0056  0.0039]
                    [ 0.0011 -0.0084]]

        왜 작은 값으로 초기화할까?
        너무 큰 값이면 출력값이 급격히 커지고, 기울기 소실(Vanishing Gradient) 문제 발생.
        너무 작은 값이면 학습이 제대로 진행되지 않음.
        일반적으로 0에 가까운 작은 정규분포 난수를 사용하여 학습이 원활하게 진행되도록 함.
        """
        self.W = np.random.randn(input_size, output_size) * 0.01  # 가중치 초기화 (작은 난수값)

        """
        출력 뉴런 수(output_size)만큼의 편향 벡터 생성.
        초기값은 0으로 설정.
        
        왜 편향(b)은 0으로 초기화할까?
        - 편향은 입력이 0일 때도 뉴런이 활성화될 수 있도록 조정하는 역할.
        - 0으로 초기화해도 학습 과정에서 자동으로 업데이트되므로 문제없음.
        """
        self.b = np.zeros(output_size)  # 편향 초기화

    """
    순전파(Forward Propagation):
    뉴럴 네트워크에서 입력을 받아 출력을 생성하는 과정
    입력 x가 가중치 W와 곱해지고, 편향 b가 더해져 다음 층으로 전달됨.
    
    x → 입력 데이터 (배치 크기 × 입력 뉴런 수)
    W → 가중치 행렬 (입력 뉴런 수 × 출력 뉴런 수)
    b → 편향 벡터 (출력 뉴런 수)
    z → 최종 출력값
    """
    def forward(self, x):
        self.x = x  # 입력 저장 (역전파에서 사용)

        z = np.dot(x, self.W) + self.b  # 행렬 곱 연산 수행
        return z


    """
    역전파(Backpropagation)란?
    뉴럴 네트워크에서 오차(Error)를 역으로 전달하여 가중치(W)와 편향(b)을 업데이트하는 과정.
    최종 출력층에서 오차를 계산한 후, 각 레이어를 거쳐 입력층 방향으로 전파됨.
    기울기(Gradient)를 계산하여 가중치 업데이트에 활용.
    """
    def backward(self, dout):
        """
        입력값 x(Transposed)와 역전파된 오차 dout를 행렬 곱 연산하여 기울기 dW를 계산.
        즉, W에 대해 손실 함수의 기울기를 구하는 과정.
        example)
        # 순전파에서 저장 (위 forward 함수 참고, `self.x = x  # 입력 저장 (역전파에서 사용)`)
        x = np.array([[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12]])  # (3,4) 입력 데이터

        dout = np.array([[1, 1],
                         [2, 2],
                         [3, 3]])  # (3,2) 오차값

        dW = np.dot(x.T, dout)  # (4,3) × (3,2) = (4,2)
        """
        self.dW = np.dot(self.x.T, dout)  # 가중치에 대한 그래디언트 계산

        """
        편향(b)에 대한 그래디언트(기울기) 계산.
        dout의 모든 샘플에 대한 오차를 축 axis=0 방향으로 합산하여 db를 구함.
        
        example)
        dout = np.array([
                            [1, 1], 
                            [2, 2], 
                            [3, 3]
                        ])  # (3,2)

        db = np.sum(dout, axis=0)  # (2,) 크기의 벡터 생성
        print(db)  # 출력: [6 6]
        """
        self.db = np.sum(dout, axis=0)  # 편향에 대한 그래디언트 계산
        return np.dot(dout, self.W.T)  # 이전 레이어로 그래디언트 전파

### 다층 퍼셉트론 (MLP) 모델 정의


In [15]:
class MLP:
    def __init__(self, input_size=784, hidden1=128, hidden2=64, hidden3=32, output_size=10):
        # 3개의 은닉층과 출력층 정의
        self.layer1 = FCLayer(input_size, hidden1)
        self.layer2 = FCLayer(hidden1, hidden2)
        self.layer3 = FCLayer(hidden2, hidden3)
        self.layer4 = FCLayer(hidden3, output_size)
        # 손실 함수 (교차 엔트로피): 다중 클래스 분류에서 사용
        self.loss_layer = lambda y, t: -np.sum(t * np.log(y + 1e-7)) / len(y)

    def predict(self, x):
        """
        순전파 (Forward Propagation)
        모델이 입력값을 받아 예측을 수행하는 과정(순전파)을 정의합니다.
        ➡ 입력 x가 네 개의 완전 연결(Fully Connected, FC) 레이어를 통과하며 변환됩니다.
        :param x: batch_size 만큼의 이미지(행)를 받음, 하나의 이미지는 28*28 사이즈 이므로, 784열을 가지고 있음
        :return:
        """
        z1 = self.layer1.forward(x)
        self.a1 = sigmoid(z1)  # 은닉층 1 활성화

        z2 = self.layer2.forward(self.a1)
        self.a2 = sigmoid(z2)  # 은닉층 2 활성화

        z3 = self.layer3.forward(self.a2)
        self.a3 = sigmoid(z3)  # 은닉층 3 활성화

        z4 = self.layer4.forward(self.a3)
        self.a4 = sigmoid(z4)  # 출력층 활성화

        return self.a4


    def backward(self, dout):
        # 출력층 역전파
        delta4 = dout * sigmoid_prime(self.a4)  # 출력층의 오차 (∂L/∂a4)
        dout3 = self.layer4.backward(delta4)  # Layer 4 역전파 수행

        # 은닉층 3 역전파
        delta3 = dout3 * sigmoid_prime(self.a3)  # ∂L/∂a3
        dout2 = self.layer3.backward(delta3)  # Layer 3 역전파 수행

        # 은닉층 2 역전파
        delta2 = dout2 * sigmoid_prime(self.a2)  # ∂L/∂a2
        dout1 = self.layer2.backward(delta2)  # Layer 2 역전파 수행

        # 은닉층 1 역전파
        delta1 = dout1 * sigmoid_prime(self.a1)  # ∂L/∂a1

        """
        sigmoid_prime(self.a4): 시그모이드 함수의 미분값을 곱하여 역전파를 위한 그래디언트 계산.
        self.layer4.backward(delta4): 가중치와 편향에 대한 그래디언트 계산 후, 이전 레이어로 그래디언트 전달.
        모든 레이어를 거쳐 최종적으로 입력층 방향으로 오차 전파 완료
        """
        self.layer1.backward(delta1)  # Layer 1 역전파 수행
    
    """
    역전파(Backpropagation) 과정에서 계산된 기울기(Gradient)를 사용하여 가중치(W)와 편향(b)을 업데이트하는 역할
    
    > self.layer1.dW: 역전파 과정에서 계산된 가중치(W)의 기울기(Gradient).
    > lr(학습률, Learning Rate): 가중치를 업데이트할 속도를 조절하는 하이퍼파라미터.
    > W -= lr * dW: 기울기 방향으로 가중치를 조정하여 손실을 줄이는 방향으로 학습.
    """
    def update(self, lr):
        # 가중치 업데이트 (경사 하강법 적용)
        self.layer1.W -= lr * self.layer1.dW
        self.layer1.b -= lr * self.layer1.db
        self.layer2.W -= lr * self.layer2.dW
        self.layer2.b -= lr * self.layer2.db
        self.layer3.W -= lr * self.layer3.dW
        self.layer3.b -= lr * self.layer3.db
        self.layer4.W -= lr * self.layer4.dW
        self.layer4.b -= lr * self.layer4.db

In [16]:

# 모델 학습 함수
def train(model: MLP, X_train, y_train, epochs=2, batch_size=125, lr=0.1):
    for epoch in range(epochs):
        """
        np.random.permutation(n): 0부터 n-1까지의 정수를 랜덤하게 섞은 배열을 반환합니다.
        len(X_train): 훈련 데이터셋의 샘플 개수(데이터 개수). 
        MNIST 데이터셋의 60000개 입력됨
        """
        idx = np.random.permutation(len(X_train))

        """
        훈련 데이터셋을 idx에 따라 무작위로 섞는 역할을 합니다.
        X_train[idx]: 기존 X_train을 idx 순서대로 다시 정렬 → X_train의 순서가 랜덤하게 섞임
        y_train[idx]: y_train도 같은 방식으로 섞어 X_train과 라벨이 일치하도록 유지
        
        데이터를 랜덤하게 섞으면 모델이 다양한 데이터 패턴을 학습할 수 있어 일반화 성능이 향상 기대
        """
        X_train, y_train = X_train[idx], y_train[idx]

        """
        i를 0부터 len(X_train)까지 batch_size 간격으로 증가시키며 반복.
        batch_size=5라면, 첫 번째 루프에서 i=0, 두 번째 루프에서 i=5, 세 번째 루프에서 i=10 … 이런 식으로 진행.
        
        Mini-batch
        X_train과 y_train을 batch_size만큼 나누어 학습하는 방법
        """
        for i in range(0, len(X_train), batch_size):
            """
            log 의 흐름 batch_size 가 "125" 일 경우
            Batch from 0 to 125
            Batch from 125 to 250 ... 이런식으로 출력됨 
            """
            # print(f"Batch from {i} to {i + batch_size}:")
            X_batch, y_batch = X_train[i:i + batch_size], y_train[i:i + batch_size]

            """
            현재 배치(X_batch)를 모델에 입력하여 예측값(y_pred)을 얻음.
            """
            y_pred = model.predict(X_batch)

            """
            예측값 y_pred와 정답 y_batch를 비교하여 손실(loss)을 계산.
            loss_layer()는 교차 엔트로피 손실을 사용
            """
            loss = model.loss_layer(y_pred, y_batch)  # 손실 계산

            """
            역전파(Backpropagation) 과정 수행.
            y_pred - y_batch → 예측값과 실제 값의 차이(오차)를 구함.
            backward() 함수를 실행하여 모델의 가중치(W)와 편향(B)에 대한 그래디언트(기울기)를 계산.
            """
            model.backward(y_pred - y_batch)  # 역전파 수행

            """
            경사 하강법(Gradient Descent)을 사용하여 가중치를 업데이트.
            """
            model.update(lr)  # 가중치 업데이트

        print(f"Epoch {epoch + 1}/{epochs} Loss: {loss:.4f}")  # 매 Epoch마다 손실 출력


# MLP 모델 생성 및 학습 수행

In [17]:
mlp = MLP()
train(mlp, X_train, y_train, batch_size=125, epochs=2, lr=0.1)

Batch from 0 to 125:
Batch from 125 to 250:
Batch from 250 to 375:
Batch from 375 to 500:
Batch from 500 to 625:
Batch from 625 to 750:
Batch from 750 to 875:
Batch from 875 to 1000:
Batch from 1000 to 1125:
Batch from 1125 to 1250:
Batch from 1250 to 1375:
Batch from 1375 to 1500:
Batch from 1500 to 1625:
Batch from 1625 to 1750:
Batch from 1750 to 1875:
Batch from 1875 to 2000:
Batch from 2000 to 2125:
Batch from 2125 to 2250:
Batch from 2250 to 2375:
Batch from 2375 to 2500:
Batch from 2500 to 2625:
Batch from 2625 to 2750:
Batch from 2750 to 2875:
Batch from 2875 to 3000:
Batch from 3000 to 3125:
Batch from 3125 to 3250:
Batch from 3250 to 3375:
Batch from 3375 to 3500:
Batch from 3500 to 3625:
Batch from 3625 to 3750:
Batch from 3750 to 3875:
Batch from 3875 to 4000:
Batch from 4000 to 4125:
Batch from 4125 to 4250:
Batch from 4250 to 4375:
Batch from 4375 to 4500:
Batch from 4500 to 4625:
Batch from 4625 to 4750:
Batch from 4750 to 4875:
Batch from 4875 to 5000:
Batch from 5000 t